Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:

#====THIS BLOCK ERRORS OUT RUN THIS BLOCK ONCE THEN RERUN BLOCKS AFTER THIS BLOCK#
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U -q "scipy>=1.11" "soundfile>=0.12"

# HF stack compatible with Tortoise (works on Colab Py3.12)
!pip install -U -q "transformers<5,>=4.28" "tokenizers<0.20" "sentencepiece>=0.1.99"

# Clone your chosen fork (jnordberg)
!git clone --depth 1 https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts

# Constrain librosa/resampy so pip won't backtrack to 0.9.x (which pulls numba/llvmlite)
!printf "librosa>=0.10,<0.11\nresampy>=0.4.3\n" > /tmp/tortoise_constraints.txt

# Install requirements under constraints + extras
!pip3 install -q -r requirements.txt -c /tmp/tortoise_constraints.txt
!pip3 install -q einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5

# Patch LogitsWarper import robustly (both markers are within the same shell command)
!cat <<'PY' > /tmp/patch_typical_sampling.py
from pathlib import Path
import re
p = Path("tortoise/utils/typical_sampling.py")
s = p.read_text(encoding="utf-8")
patched = re.sub(
    r"^\s*from\s+transformers\s+import\s+LogitsWarper\s*$",
    "try:\\n    from transformers import LogitsWarper\\n"
    "except Exception:\\n"
    "    try:\\n"
    "        from transformers.generation.logits_process import LogitsWarper\\n"
    "    except Exception:\\n"
    "        class LogitsWarper:\\n"
    "            def __call__(self, input_ids, scores):\\n"
    "                raise NotImplementedError('LogitsWarper unavailable in this transformers build.')",
    s, flags=re.MULTILINE)
if patched != s:
    p.write_text(patched, encoding="utf-8")
    print(f"Patched {p}")
else:
    print("No patch needed.")

!python3 /tmp/patch_typical_sampling.py

# Proper install for a pyproject/PEP 517 repo (do NOT run setup.py)
!pip3 install -q -e .

# Smoke test (both heredoc markers kept inside the same shell command)
!python3 - <<'PY'
from tortoise.api import TextToSpeech
print("[OK] tortoise import")
tts = TextToSpeech()
print("[OK] Tortoise TextToSpeech ready")

[Errno 2] No such file or directory: 'tortoise-tts'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reason for being yanked: wrong version of the code was released in the .whl package
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.5/236.5 kB 21.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Faile

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

...

Done.


Done.


Done.


Done.


RuntimeError: Error(s) in loading state_dict for UnifiedVoice:
	Unexpected key(s) in state_dict: "gpt.h.0.attn.bias", "gpt.h.0.attn.masked_bias", "gpt.h.1.attn.bias", "gpt.h.1.attn.masked_bias", "gpt.h.2.attn.bias", "gpt.h.2.attn.masked_bias", "gpt.h.3.attn.bias", "gpt.h.3.attn.masked_bias", "gpt.h.4.attn.bias", "gpt.h.4.attn.masked_bias", "gpt.h.5.attn.bias", "gpt.h.5.attn.masked_bias", "gpt.h.6.attn.bias", "gpt.h.6.attn.masked_bias", "gpt.h.7.attn.bias", "gpt.h.7.attn.masked_bias", "gpt.h.8.attn.bias", "gpt.h.8.attn.masked_bias", "gpt.h.9.attn.bias", "gpt.h.9.attn.masked_bias", "gpt.h.10.attn.bias", "gpt.h.10.attn.masked_bias", "gpt.h.11.attn.bias", "gpt.h.11.attn.masked_bias", "gpt.h.12.attn.bias", "gpt.h.12.attn.masked_bias", "gpt.h.13.attn.bias", "gpt.h.13.attn.masked_bias", "gpt.h.14.attn.bias", "gpt.h.14.attn.masked_bias", "gpt.h.15.attn.bias", "gpt.h.15.attn.masked_bias", "gpt.h.16.attn.bias", "gpt.h.16.attn.masked_bias", "gpt.h.17.attn.bias", "gpt.h.17.attn.masked_bias", "gpt.

In [ ]:
%cat /content/tortoise-tts/tortoise-tts/requirements.txt

cat: /content/tortoise-tts/tortoise-tts/requirements.txt: No such file or directory


In [ ]:
# /notebooks/voice_preprocess_split.py
# Replace your current upload cell with this one.

# Minimal deps for robust audio IO in Colab
!pip -q install librosa soundfile  >/dev/null

from __future__ import annotations
import os
import re
import io
from pathlib import Path
from typing import Dict, Tuple, List

import numpy as np
import librosa
import soundfile as sf
from google.colab import files

# --- Config (edit as needed) ---
CUSTOM_VOICE_NAME: str = "martin"
TARGET_SR: int = 22_050         # Tortoise-friendly SR
CHUNK_SECONDS: float = 10.0     # exact chunk length
TOP_DB: float = 30.0            # silence trim threshold
PEAK_TARGET: float = 0.89       # ≈ -1 dBFS
MIN_KEEP_SECONDS: float = 3.0   # drop leftover if shorter than this
PAD_LAST_SEGMENT: bool = True   # pad last chunk with silence to exact length

# --- Helpers ---
def _scan_next_index(folder: Path) -> int:
    patt = re.compile(r'^(\d+)\.wav$', re.IGNORECASE)
    idxs = []
    for name in os.listdir(folder):
        m = patt.match(name)
        if m:
            idxs.append(int(m.group(1)))
    return (max(idxs) + 1) if idxs else 0

def _peak_normalize(y: np.ndarray, target_peak: float) -> np.ndarray:
    peak = float(np.max(np.abs(y))) if y.size else 0.0
    if peak == 0.0:
        return y
    scale = min(target_peak / peak, 1.0)  # avoid upscaling above target
    return (y * scale).astype(np.float32)

def _load_mono_resample(temp_path: Path) -> Tuple[np.ndarray, int]:
    # librosa handles a wide range; mono=True + resample to TARGET_SR
    y, sr = librosa.load(str(temp_path), sr=TARGET_SR, mono=True)
    return y.astype(np.float32), TARGET_SR

def _trim_silence(y: np.ndarray, top_db: float) -> np.ndarray:
    if y.size == 0:
        return y
    yt, _ = librosa.effects.trim(y, top_db=top_db)
    return yt

def _split_into_chunks(y: np.ndarray, sr: int, chunk_sec: float) -> List[np.ndarray]:
    chunk_len = int(round(chunk_sec * sr))
    if y.size == 0:
        return []
    chunks = []
    for start in range(0, len(y), chunk_len):
        end = start + chunk_len
        seg = y[start:end]
        if len(seg) < chunk_len:
            if len(seg) < int(MIN_KEEP_SECONDS * sr) and not PAD_LAST_SEGMENT:
                continue
            if PAD_LAST_SEGMENT:
                pad = np.zeros(chunk_len - len(seg), dtype=seg.dtype)  # keep exact length
                seg = np.concatenate([seg, pad], axis=0)
        chunks.append(seg)
    return chunks

def _save_chunks(chunks: List[np.ndarray], sr: int, folder: Path, start_idx: int) -> int:
    written = 0
    idx = start_idx
    for seg in chunks:
        if len(seg) < int(MIN_KEEP_SECONDS * sr):
            # skip too-short segments (quality safeguard)
            continue
        fname = f"{idx:03d}.wav"
        out_path = folder / fname
        # 16-bit PCM WAV; safer for Tortoise dataset ingestion
        sf.write(out_path, seg, sr, subtype="PCM_16")
        idx += 1
        written += 1
    return written

# --- Main ---
voice_folder = Path("tortoise/voices") / CUSTOM_VOICE_NAME
voice_folder.mkdir(parents=True, exist_ok=True)

print(f"Upload long-form audio (any common format). We'll trim, normalize, and split into exact {CHUNK_SECONDS:.0f}s WAVs…")
uploaded: Dict[str, bytes] = files.upload()  # user selects 1+ files

total_written = 0
next_idx = _scan_next_index(voice_folder)
report_rows = []

for i, (name, data) in enumerate(uploaded.items()):
    # Keep original extension for temp decoding
    suffix = Path(name).suffix or ".tmp"
    tmp_path = Path("/tmp") / f"cc_upload_{i}{suffix}"
    with open(tmp_path, "wb") as f:
        f.write(data)

    # Decode → mono 22.05 kHz
    y, sr = _load_mono_resample(tmp_path)

    # Trim leading/trailing silence to reduce dead air
    y = _trim_silence(y, TOP_DB)

    # Normalize near -1 dBFS to avoid clipping across chunks
    y = _peak_normalize(y, PEAK_TARGET)

    # Chunk and (optionally) pad last to exact 10s
    chunks = _split_into_chunks(y, sr, CHUNK_SECONDS)

    # Persist as sequentially numbered WAVs
    written = _save_chunks(chunks, sr, voice_folder, next_idx)
    total_written += written
    next_idx += written
    duration_min = (len(y) / sr) / 60 if sr else 0
    report_rows.append((name, len(chunks), written, f"{duration_min:.2f} min"))

# Summary
print(f"\nSaved {total_written} WAV chunk(s) into: {voice_folder}")
for name, made, kept, dur in report_rows:
    print(f" - {name}: made {made}, kept {kept}, source ~{dur}")
print("Examples:", *(sorted(str(p) for p in voice_folder.glob('*.wav'))[:3]), sep="\n - ")

Upload long-form audio (any common format). We'll trim, normalize, and split into exact 10s WAVs…


Saving Voice 250930_172523.m4a to Voice 250930_172523.m4a


/tmp/ipython-input-926286703.py:47: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(str(temp_path), sr=TARGET_SR, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



Saved 4 WAV chunk(s) into: tortoise/voices/martin
 - Voice 250930_172523.m4a: made 4, kept 4, source ~0.53 min
Examples:
 - tortoise/voices/martin/000.wav
 - tortoise/voices/martin/001.wav
 - tortoise/voices/martin/002.wav


In [ ]:
# This is the text that will be spoken.
text = 'put words here'

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
# === Safe Tortoise imports + init (fixes "Unexpected key(s) in state_dict") ===
import re
import contextlib
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import IPython

# --- Transformers LogitsWarper compat (safe no-op if already OK) ---
try:
    from transformers import LogitsWarper  # noqa: F401
except Exception:
    try:
        from transformers.generation.logits_process import LogitsWarper as _LW  # HF 4.x path
        import transformers as _tf; setattr(_tf, "LogitsWarper", _LW)
    except Exception:
        class LogitsWarper:  # only to satisfy inheritance if neither path exists
            def __call__(self, input_ids, scores):
                raise NotImplementedError("LogitsWarper unavailable in this transformers build.")

# --- Scoped patches (WHY: limit impact to model load; avoid recursion across cells) ---
@contextlib.contextmanager
def patch_torch_load_strip_attn():
    """Strip legacy GPT attention buffer keys that newer modules don't register."""
    orig = torch.load
    pat = re.compile(r"\bgpt\.h\.\d+\.attn\.(?:bias|masked_bias)\b")
    def _patched(*args, **kwargs):
        obj = orig(*args, **kwargs)
        if isinstance(obj, dict):
            if "state_dict" in obj and isinstance(obj["state_dict"], dict):
                sd = obj["state_dict"]
                dels = [k for k in list(sd) if pat.search(k)]
                for k in dels: del sd[k]
                if dels: print(f"[torch.load] Stripped {len(dels)} legacy attn buffer keys (nested).")
            else:
                dels = [k for k in list(obj) if isinstance(k, str) and pat.search(k)]
                for k in dels: del obj[k]
                if dels: print(f"[torch.load] Stripped {len(dels)} legacy attn buffer keys (flat).")
        return obj
    torch.load = _patched
    try:
        yield
    finally:
        torch.load = orig

@contextlib.contextmanager
def patch_load_state_dict_tolerant():
    """Retry with strict=False only when 'Unexpected key(s)' occurs."""
    orig = torch.nn.Module.load_state_dict
    def wrapper(self, state_dict, *args, **kwargs):
        try:
            return orig(self, state_dict, *args, **kwargs)
        except RuntimeError as e:
            if "Unexpected key(s) in state_dict" in str(e) and kwargs.get("strict", True):
                kw = dict(kwargs); kw["strict"] = False
                print(f"[load_state_dict] {self.__class__.__name__}: strict=False fallback.")
                return orig(self, state_dict, *args, **kw)
            raise
    torch.nn.Module.load_state_dict = wrapper
    try:
        yield
    finally:
        torch.nn.Module.load_state_dict = orig

# --- Import Tortoise AFTER the shims (safe either way, but clearer) ---
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# --- Initialize TTS with scoped patches applied ---
with patch_load_state_dict_tolerant(), patch_torch_load_strip_attn():
    tts = TextToSpeech()
print("Tortoise TextToSpeech ready.")

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

[torch.load] Stripped 60 legacy attn buffer keys (flat).


/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Tortoise TextToSpeech ready.


In [ ]:
# /notebooks/tortoise_synthesize_cvvp_fallback.py
# Run after your upload/split + TTS init cell ("Tortoise TextToSpeech ready.")

from pathlib import Path
import time
import torch
import torchaudio
from IPython.display import Audio, display

# ---- Config (reuses your globals if already set) ----
CUSTOM_VOICE_NAME = globals().get("CUSTOM_VOICE_NAME", "martin")
text = globals().get("text", "Hello, this is a quick test of my cloned voice using Tortoise.")
PRESET = globals().get("PRESET", "fast")          # 'ultra_fast' | 'fast' | 'standard' | 'high_quality'
SEED = globals().get("SEED", None)                # set an int for determinism
CVVP_AMOUNT = globals().get("CVVP_AMOUNT", 0.0)   # may be unsupported in your build

OUT_SR = 24_000
OUTPUT_DIR = Path("tortoise/outputs") / CUSTOM_VOICE_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ---- Ensure TTS + voice loaded (no re-patching here) ----
try:
    tts  # noqa: F821
except NameError:
    from tortoise.api import TextToSpeech
    tts = TextToSpeech()

try:
    voice_samples  # noqa: F821
    conditioning_latents  # noqa: F821
except NameError:
    from tortoise.utils.audio import load_voice
    voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

# ---- Helpers ----
def _ensure_mono_2d(x: torch.Tensor) -> torch.Tensor:
    """Why: torchaudio.save expects [C, T]."""
    x = x.detach().cpu()
    if x.ndim == 1: return x.unsqueeze(0)
    if x.ndim == 2: return x.mean(0, keepdim=True) if x.size(0) > 1 else x
    if x.ndim == 3: return _ensure_mono_2d(x[0])
    raise ValueError(f"Unexpected audio tensor shape: {tuple(x.shape)}")

def synth_with_cvvp_fallback(txt: str):
    base_kwargs = dict(
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset=PRESET,
        seed=SEED,
    )
    # First try with cvvp_amount
    try:
        return tts.tts_with_preset(txt, **base_kwargs, cvvp_amount=CVVP_AMOUNT)
    except ValueError as e:
        if "cvvp_amount" in str(e):
            print("[warn] 'cvvp_amount' unsupported in this build. Retrying without it.")
            return tts.tts_with_preset(txt, **base_kwargs)
        raise

# ---- Synthesize + Save ----
gen = synth_with_cvvp_fallback(text)
wav = _ensure_mono_2d(gen.squeeze())
out_path = OUTPUT_DIR / f"generated-{CUSTOM_VOICE_NAME}.wav"
torchaudio.save(str(out_path), wav, OUT_SR)
print(f"Wrote: {out_path}")
display(Audio(str(out_path), rate=OUT_SR))

Generating autoregressive samples..


  0%|          | 0/6 [00:00<?, ?it/s]

[warn] 'cvvp_amount' unsupported in this build. Retrying without it.


Generating autoregressive samples..


100%|██████████| 6/6 [00:28<00:00,  4.69s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 6/6 [00:09<00:00,  1.61s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:05<00:00, 15.82it/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:247: UserWarning: torio.io._streaming_media_encoder.StreamingMediaEncoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information

Wrote: tortoise/outputs/martin/generated-martin.wav


In [ ]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install